In [ ]:
# Library used
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image
import pydotplus

In [ ]:
# Function to import csv data from url
def importdata(url):
    product_df = pd.read_csv(
        url)
    # Displaying dataset information
    print("Dataset Length: ", len(product_df))
    print("Dataset Shape: ", product_df.shape)
    print("Dataset: \n", product_df.head())

    return product_df

In [ ]:
# Function for splitting dataset into 70% train and 30% test
def splitdataset(balance_data):
    # Separating the targe variable
    X = balance_data.iloc[:, :-1]
    Y = balance_data["CA level"].values
    
    # Checking if X and Y are string before encoding
    isXString = False
    isYString = False
    for i in X:
        if isinstance(i, str):
            isXString = True
    for i in Y:
        if isinstance(i, str):
            isYString = True
    
    # Label Encoding
    if isXString:
        le = LabelEncoder()
        X = X.apply(le.fit_transform)
    if isYString:
        le = LabelEncoder()
        Y = Y.apply(le.fit_transform)
    # Splitting the dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(
        X, Y, test_size=0.3, random_state=100
    )
    
    return X, Y, X_train, X_test, y_train, y_test

In [ ]:
# Function to perform training algorithms

# Using Gini Index
def train_using_gini(X_train, X_test, y_train):
    
    # Creating the classifier object
    clf_gini = DecisionTreeClassifier(criterion="gini",
                                      random_state=100, class_weight='balanced')#, max_depth=3, min_samples_leaf=5)
    # Performing training
    clf_gini.fit(X_train, y_train)
    return clf_gini

# Using Entropy
def train_using_entropy(X_train, X_test, y_train):
    # Creating the classifier object
    clf_entropy = DecisionTreeClassifier(criterion="entropy",
                                      random_state=100, class_weight='balanced')#, max_depth=3, min_samples_leaf=5)
    # Performing training
    clf_entropy.fit(X_train, y_train)
    return clf_entropy

In [ ]:
# Function to make predictions
def prediction(X_test, clf_object):
    y_pred = clf_object.predict(X_test)
    print("Predicted values:", y_pred)
    return y_pred

In [ ]:
# Placeholder function for cal_accuracy
def cal_accuracy(y_test, y_pred):
    print("Confusion Matrix: ",
          metrics.confusion_matrix(y_test, y_pred))
    print("Accuracy : ",
          metrics.accuracy_score(y_test, y_pred)*100)
    print("Report : ",
          metrics.classification_report(y_test, y_pred))

In [ ]:
# Driver code
url = "cancer.csv"
data = importdata(url)
X, Y, X_train, X_test, y_train, y_test = splitdataset(data)

In [ ]:
# Training using gini and entropy
clf_gini = train_using_gini(X_train, X_test, y_train)
clf_entropy = train_using_entropy(X_train, X_test, y_train)

In [ ]:
# Visualize decision tree

dot_data = StringIO()
export_graphviz(clf_gini, out_file=dot_data,
                filled=True, rounded=True,
                special_characters=True, feature_names=X.columns, class_names=['A','B','C','D'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png('cancer_gini.png')
Image(graph.create_png())

dot_data = StringIO()
export_graphviz(clf_entropy, out_file=dot_data,
                filled=True, rounded=True,
                special_characters=True, feature_names=X.columns, class_names=['A','B','C','D'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png('cancer_entropy.png')
Image(graph.create_png())

In [ ]:
# Operational Phase
print("Results Using Gini Index:")
y_pred_gini = prediction(X_test, clf_gini)
cal_accuracy(y_test, y_pred_gini)

print("Results Using Entropy:")
y_pred_entropy = prediction(X_test, clf_entropy)
cal_accuracy(y_test, y_pred_entropy)